In [1]:
import numpy as np
import random
from qiskit import QuantumCircuit, ClassicalRegister, transpile
from qiskit.transpiler import PassManager, TransformationPass
from qiskit.dagcircuit import DAGCircuit, DAGOpNode

from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_aer.primitives import SamplerV2 as AerSampler

# Use qiskit_ibm_runtime.fake_provider for modern backends
from qiskit_ibm_runtime.fake_provider import FakeAthensV2

# Load the backend
backend = FakeAthensV2()


In [2]:
def sample_circuit(circuit, backend, shots=None):
        """
        Sample a quantum circuit and return the probability distribution.
        
        Assumes the input circuit is already transpiled.
        
        If self.twirl is True, this returns the *averaged* distribution
        from many randomly twirled circuits.
        
        If self.twirl is False, this returns the distribution from a
        single run of the original circuit.
        """
        if shots is None:
            shots = 2048
            
        n_qubits = circuit.num_qubits
        final_distribution = np.zeros(2**n_qubits)

        circuit_with_meas = circuit.copy()
        if not circuit_with_meas.cregs:
            circuit_with_meas.add_register(ClassicalRegister(n_qubits, 'meas'))
        circuit_with_meas.measure_all()
        
        job = backend.run(circuit_with_meas, shots=shots)
        result = job.result()
        counts = result.get_counts()

        for state_str, count in counts.items():
            clean_state_str = state_str.replace(' ', '')
            if len(clean_state_str) > n_qubits:
                 clean_state_str = clean_state_str[:n_qubits]
                 
            state_int = int(clean_state_str, 2)
            if state_int < len(final_distribution):
                if shots > 0:
                    final_distribution[state_int] = count / shots
                else:
                    final_distribution[state_int] = 0.0

        return final_distribution

In [3]:
def random_STHCZ_circuit(n_qubits: int, depth_layers: int, p_cz=0.5):
    qc = QuantumCircuit(n_qubits)
    oneq_gates = ["h","s","t"]
    for _ in range(depth_layers):
        for q in range(n_qubits):
            g = np.random.choice(oneq_gates)
            getattr(qc, g)(q)
        if np.random.rand() < p_cz and n_qubits >= 2:
            a, b = np.random.choice(n_qubits, size=2, replace=False)
            qc.cz(a, b)
        qc.barrier()
    return qc

Now do Pauli twirling

In [4]:
from qiskit.circuit.library import IGate, XGate, YGate, ZGate
import random
from qiskit.transpiler import PassManager, TransformationPass
from qiskit.dagcircuit import DAGCircuit

# --- Define Pauli gates and conjugation lookup tables ---

PAULIS = {'I': IGate(), 'X': XGate(), 'Y': YGate(), 'Z': ZGate()}
PAULI_NAMES = ['I', 'X', 'Y', 'Z']

# ---
# These are the correct maps, verified by P_out = G * P_in * G_dagger
# (ignoring phase)
# ---

# P_out = X * P_in * X
# X*Y*X = -Y, X*Z*X = -Z
X_CONJUGATION_MAP = {'I': 'I', 'X': 'X', 'Y': 'Y', 'Z': 'Z'}

# P_out = SX * P_in * SX_dagger
# SX*X*SX_dag = X
# SX*Y*SX_dag = Z
# SX*Z*SX_dag = -Y
SX_CONJUGATION_MAP = {'I': 'I', 'X': 'X', 'Y': 'Z', 'Z': 'Y'} 

# P_out = CNOT * P_in * CNOT
CNOT_CONJUGATION_MAP = {
    'II': ('I', 'I'), 'IX': ('I', 'X'), 'IY': ('Z', 'Y'), 'IZ': ('Z', 'Z'),
    'XI': ('X', 'X'), 'XX': ('X', 'I'), 'XY': ('Y', 'Z'), 'XZ': ('Y', 'Y'),
    'YI': ('Y', 'X'), 'YX': ('Y', 'I'), 'YY': ('X', 'Z'), 'YZ': ('X', 'Y'),
    'ZI': ('Z', 'I'), 'ZX': ('Z', 'X'), 'ZY': ('I', 'Y'), 'ZZ': ('I', 'Z'),
}

class ManualPauliTwirl(TransformationPass):
    """
    Randomly inserts Pauli gates around 'cx', 'sx', and 'x' gates
    to twirl the noise channels.
    
    This pass should be run ON A TRANSPILED CIRCUIT.
    """
    def __init__(self, seed=None):
        super().__init__()
        self.rng = random.Random(seed)

    def run(self, dag: DAGCircuit) -> DAGCircuit:
        new_dag = dag.copy_empty_like()
        
        for node in dag.topological_op_nodes():
            if node.op.name == 'cx':
                # --- Twirl CNOT ---
                p_c_in_str = self.rng.choice(PAULI_NAMES)
                p_t_in_str = self.rng.choice(PAULI_NAMES)
                p_in_key = p_c_in_str + p_t_in_str
                p_c_out_str, p_t_out_str = CNOT_CONJUGATION_MAP[p_in_key]
                
                if p_c_in_str != 'I': new_dag.apply_operation_back(PAULIS[p_c_in_str], [node.qargs[0]])
                if p_t_in_str != 'I': new_dag.apply_operation_back(PAULIS[p_t_in_str], [node.qargs[1]])
                new_dag.apply_operation_back(node.op, node.qargs, node.cargs)
                if p_c_out_str != 'I': new_dag.apply_operation_back(PAULIS[p_c_out_str], [node.qargs[0]])
                if p_t_out_str != 'I': new_dag.apply_operation_back(PAULIS[p_t_out_str], [node.qargs[1]])
            
            elif node.op.name == 'sx':
                # --- Twirl SX ---
                p_in_str = self.rng.choice(PAULI_NAMES)
                p_out_str = SX_CONJUGATION_MAP[p_in_str]
                
                if p_in_str != 'I': new_dag.apply_operation_back(PAULIS[p_in_str], node.qargs)
                new_dag.apply_operation_back(node.op, node.qargs, node.cargs)
                if p_out_str != 'I': new_dag.apply_operation_back(PAULIS[p_out_str], node.qargs)
                    
            elif node.op.name == 'x':
                # --- Twirl X ---
                p_in_str = self.rng.choice(PAULI_NAMES)
                p_out_str = X_CONJUGATION_MAP[p_in_str]
                
                if p_in_str != 'I': new_dag.apply_operation_back(PAULIS[p_in_str], node.qargs)
                new_dag.apply_operation_back(node.op, node.qargs, node.cargs)
                if p_out_str != 'I': new_dag.apply_operation_back(PAULIS[p_out_str], node.qargs)
                    
            elif node.op.name not in ['barrier', 'measure']:
                # --- Keep all other gates (like rz, id) ---
                new_dag.apply_operation_back(node.op, node.qargs, node.cargs)

        return new_dag

In [5]:
def classically_flip_distribution(distribution, n_qubits, flip_mask_array):
    """
    Classically flips the bits of a probability distribution array.
    
    Args:
        distribution (np.ndarray): The 2^n array of probabilities.
        n_qubits (int): The total number of qubits (e.g., 5 for Athens).
        flip_mask_array (list[int]): A list of 0s and 1s, e.g., [0, 1, 0, 0, 1].
                                     (Must be length n_qubits)
    
    Returns:
        np.ndarray: A new distribution with probabilities re-mapped.
    """
    
    # Convert the mask array [0, 1, 0, 1] into a single integer (e.g., 5)
    # We reverse the list for Qiskit's little-endian bit ordering (q4,q3,q2,q1,q0)
    mask_str = "".join(map(str, flip_mask_array[::-1]))
    mask_int = int(mask_str, 2)
    
    new_distribution = np.zeros_like(distribution)
    
    for state_int, prob in enumerate(distribution):
        if prob > 0:
            # This is the classical bit-flip
            flipped_state_int = state_int ^ mask_int 
            if flipped_state_int < len(new_distribution):
                new_distribution[flipped_state_int] = prob
            
    return new_distribution

In [6]:
def get_twirled_distribution(circuit, backend, total_shots=8192, num_randomizations=100):
    """
    Generates the average probability distribution from N twirled circuits.
    
    Args:
        circuit (QuantumCircuit): The *transpiled* circuit to twirl.
        backend (Backend): The noisy backend (e.g., FakeMarrakesh).
        total_shots (int): Total shots to be divided among all randomizations.
        num_randomizations (int): Number of twirled circuits to average (N).
        
    Returns:
        np.ndarray: The final, averaged probability distribution.
    """
    
    if total_shots < num_randomizations:
        print(f"Warning: total_shots ({total_shots}) < num_randomizations ({num_randomizations}).")
        print("Running each randomization with 1 shot.")
        shots_per_randomization = 1
    else:
        shots_per_randomization = total_shots // num_randomizations
    
    n_qubits = circuit.num_qubits
    summed_distribution = np.zeros(2**n_qubits)
   
    #print(f"Running {num_randomizations} twirled randomizations with {shots_per_randomization} shots each...")
    
    for i in range(num_randomizations):
        # 1. Create a PassManager with a NEW seed for this loop
        # This generates one random twirled circuit
        twirl_pass = ManualPauliTwirl(seed=i)
        pm = PassManager([twirl_pass])
        qc_twirled = pm.run(circuit)
        
        # 2. Run this single twirled circuit
        # We use your existing, verified sample_circuit function
        dist = sample_circuit(qc_twirled, backend, shots=shots_per_randomization)
        
        # 3. Add its result to the sum
        summed_distribution += dist
        
    # 4. Average the final distribution
    averaged_distribution = summed_distribution / num_randomizations
    
    #print("Twirling complete.")
    return averaged_distribution

In [7]:
def get_full_twirled_distribution(
    logical_circuit, 
    backend, 
    basis_gates, 
    shots=8192, 
    num_gate_randomizations=100, 
    num_meas_randomizations=100
):
    """
    Performs both gate twirling and measurement twirling, then averages.
    
    This version explicitly maps logical qubits to physical qubits
    to avoid layout-checking errors.
    """
    
    n_qubits = logical_circuit.num_qubits 
    
    summed_distribution = np.zeros(2**n_qubits)
    
    shots_per_meas_run = shots

    #print(f"Running {num_meas_randomizations} measurement randomizations...")
    
    for i in range(num_meas_randomizations):
        
        qc_with_mask = logical_circuit.copy()
        
        # --- 1. ADD MEASUREMENT MASK (QUANTUM) ---
        
        # Create a random mask just for the logical qubits
        meas_flip_mask_logical = [random.choice([0, 1]) for _ in range(n_qubits)]
        
        for q_idx, flip in enumerate(meas_flip_mask_logical):
            if flip == 1:
                qc_with_mask.x(q_idx) # Flips logical qubits
        
        # --- 2. TRANSPILE (with explicit layout) ---
        
        # We create a simple layout map, e.g., [0, 1, 2...]
        # For your 1-qubit test, this will be [0]
        logical_to_physical_map = list(range(n_qubits))
        
        qc_transpiled_with_mask = transpile(
            qc_with_mask, 
            basis_gates=basis_gates,
            optimization_level=0,
            initial_layout=logical_to_physical_map # Force logical 0 -> physical 0
        )

        # --- 3. CALL GATE-TWIRLING FUNCTION ---
        
        gate_twirled_dist = get_twirled_distribution(
            qc_transpiled_with_mask, 
            backend, 
            total_shots=shots_per_meas_run, 
            num_randomizations=num_gate_randomizations
        )
        
        # --- 4. UN-FLIP (CLASSICAL) ---
        
        # Create the full 5-bit mask for the classical flip
        full_meas_mask = [0] * n_qubits
        for logical_q_idx, flip in enumerate(meas_flip_mask_logical):
            if flip == 1:
                # We know logical_q_idx maps to physical_q_idx
                physical_q_idx = logical_to_physical_map[logical_q_idx]
                full_meas_mask[physical_q_idx] = 1
        
        # Only flip if the mask is not all zeros
        if sum(full_meas_mask) > 0:
            dist_unflipped = classically_flip_distribution(
                gate_twirled_dist, 
                n_qubits, 
                full_meas_mask
            )
        else:
            dist_unflipped = gate_twirled_dist # No flip needed
        
        # --- 5. ADD TO SUM ---
        summed_distribution += dist_unflipped

    # --- 6. RETURN FINAL AVERAGE ---
    print("Full twirling complete.")
    return summed_distribution / num_meas_randomizations

In [13]:
# --- NEW WORKFLOW ---
circ = random_STHCZ_circuit(5,5)

basis_gates = ['cx', 'id', 'rz', 'sx', 'x']  # IBM basis

circ_transpiled = transpile(
            circ, 
            basis_gates=basis_gates,
        )
# 1. Create and transpile your circuit *once*
print("Original Transpiled Circuit:")
print(circ_transpiled.draw())

# 2. Get the UN-TWIRLED (non-Pauli) distribution
dist_un_twirled = sample_circuit(circ_transpiled, backend, shots=8192)
print("\n--- UN-TWIRLED Distribution (non-Pauli channel) ---")
print(dist_un_twirled)

# 3. Get the TWIRLED (Pauli-fied) distribution
dist_twirled = get_full_twirled_distribution(
    circ, 
    backend, 
    basis_gates, 
    shots=1024, 
    num_gate_randomizations=20, 
    num_meas_randomizations=20
)
print("\n--- AVERAGED TWIRLED Distribution (Pauli-fied channel) ---")
print(dist_twirled)

Original Transpiled Circuit:
global phase: 3π/4
     ┌─────────┐                  ░ ┌─────────┐                  ░ ┌─────────┐»
q_0: ┤ Rz(π/2) ├──────────────────░─┤ Rz(π/4) ├──────────────────░─┤ Rz(π/2) ├»
     ├─────────┤┌────┐┌─────────┐ ░ ├─────────┤┌────┐┌─────────┐ ░ ├─────────┤»
q_1: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░─┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░─┤ Rz(π/2) ├»
     ├─────────┤└────┘└─────────┘ ░ ├─────────┤└────┘└─────────┘ ░ ├─────────┤»
q_2: ┤ Rz(π/2) ├──────────────────░─┤ Rz(π/2) ├──────────────────░─┤ Rz(π/4) ├»
     ├─────────┤                  ░ ├─────────┤                  ░ ├─────────┤»
q_3: ┤ Rz(π/4) ├──────────────────░─┤ Rz(π/2) ├──────────────────░─┤ Rz(π/2) ├»
     ├─────────┤┌────┐┌─────────┐ ░ ├─────────┤                  ░ ├─────────┤»
q_4: ┤ Rz(π/2) ├┤ √X ├┤ Rz(π/2) ├─░─┤ Rz(π/4) ├──────────────────░─┤ Rz(π/2) ├»
     └─────────┘└────┘└─────────┘ ░ └─────────┘                  ░ └─────────┘»
«                       ░ ┌─────────┐                                   

In [14]:
dist_twirled_2 = get_full_twirled_distribution(
    circ, 
    backend, 
    basis_gates, 
    shots=1024, 
    num_gate_randomizations=20, 
    num_meas_randomizations=20
)
print("\n--- AVERAGED TWIRLED Distribution (2nd Try) ---")
print(dist_twirled_2)

Full twirling complete.

--- AVERAGED TWIRLED Distribution (2nd Try) ---
[1.19950980e-01 8.33333333e-04 1.19558824e-01 8.82352941e-04
 3.67647059e-03 4.90196078e-05 2.84313725e-03 0.00000000e+00
 1.20196078e-01 7.35294118e-04 1.21568627e-01 7.35294118e-04
 2.84313725e-03 0.00000000e+00 2.50000000e-03 9.80392157e-05
 1.25245098e-01 7.84313725e-04 1.22941176e-01 1.12745098e-03
 2.84313725e-03 0.00000000e+00 3.08823529e-03 0.00000000e+00
 1.19068627e-01 7.35294118e-04 1.20196078e-01 5.39215686e-04
 3.48039216e-03 9.80392157e-05 3.33333333e-03 4.90196078e-05]


In [15]:
dist_twirled_3 = get_full_twirled_distribution(
    circ, 
    backend, 
    basis_gates, 
    shots=1024, 
    num_gate_randomizations=20, 
    num_meas_randomizations=20
)
print("\n--- AVERAGED TWIRLED Distribution (3rd Try) ---")
print(dist_twirled_3)

Full twirling complete.

--- AVERAGED TWIRLED Distribution (3rd Try) ---
[1.18480392e-01 1.61764706e-03 1.21813725e-01 1.32352941e-03
 3.33333333e-03 9.80392157e-05 4.01960784e-03 0.00000000e+00
 1.20980392e-01 1.56862745e-03 1.22941176e-01 1.42156863e-03
 3.08823529e-03 0.00000000e+00 3.23529412e-03 9.80392157e-05
 1.20343137e-01 1.32352941e-03 1.20098039e-01 7.84313725e-04
 4.36274510e-03 0.00000000e+00 3.33333333e-03 4.90196078e-05
 1.18627451e-01 1.47058824e-03 1.18431373e-01 1.42156863e-03
 2.84313725e-03 9.80392157e-05 2.79411765e-03 0.00000000e+00]


In [11]:
# Produce Ideal distribution
ideal_backend = AerSimulator()
sample_circuit(circ_transpiled, ideal_backend, shots=8192)

array([0.03442383, 0.03381348, 0.03027344, 0.03137207, 0.03051758,
       0.03112793, 0.02661133, 0.03405762, 0.02978516, 0.03222656,
       0.03076172, 0.03186035, 0.02941895, 0.03100586, 0.03076172,
       0.02978516, 0.0333252 , 0.02905273, 0.0300293 , 0.03125   ,
       0.03588867, 0.03405762, 0.02807617, 0.02893066, 0.03466797,
       0.03186035, 0.03088379, 0.03137207, 0.02856445, 0.03186035,
       0.03088379, 0.03149414])

Test to verify whether noise is effectively Pauli after twirling (need to verify)

In [12]:
# --- Imports from your previous code ---
import numpy as np
import random
from qiskit import QuantumCircuit, ClassicalRegister, transpile
from qiskit.transpiler import PassManager
from qiskit_ibm_runtime.fake_provider import FakeAthensV2
from qiskit.circuit.library import SdgGate

# --- (Your ManualPauliTwirl class, sample_circuit, and get_twirled_distribution functions go here) ---
# ... (assuming they are defined as before) ...

# --- Helper function for this test ---
def get_marginal_expval_z(distribution, n_qubits, qubit_index):
    """
    Calculates <Z> for a single qubit from a full distribution.
    
    Args:
        distribution (np.ndarray): The 2^n array of probabilities.
        n_qubits (int): The total number of qubits (e.g., 5 for Athens).
        qubit_index (int): The physical qubit to measure (0 to n-1).
    """
    prob_0 = 0.0
    prob_1 = 0.0
    
    # This bitmask isolates the bit for our qubit_index
    # We assume Qiskit's standard bit ordering: q_n-1 ... q_index ... q_0
    bit_mask = 1 << qubit_index 
    
    for state_int, prob in enumerate(distribution):
        if (state_int & bit_mask) == 0:
            # This state has |0> for our qubit
            prob_0 += prob
        else:
            # This state has |1> for our qubit
            prob_1 += prob
            
    return prob_0 - prob_1

# --- 1. Setup the Backend and Test Circuits ---
backend = FakeAthensV2()
n_qubits_total = backend.num_qubits # 5 for FakeAthens
basis_gates = ['cx', 'id', 'rz', 'sx', 'x']

# We will test logical qubit 0. The transpiler will map it to a physical qubit.
logical_qubit = 0 

# Circuit 1: Prepare |+>, apply X, measure <X>
qc_x_meas = QuantumCircuit(1, 1)
qc_x_meas.h(0)
qc_x_meas.x(0) # Apply the noisy, twirled gate
qc_x_meas.h(0)
# Measure will be added by sample_circuit

# Circuit 2: Prepare |+>, apply X, measure <Y>
qc_y_meas = QuantumCircuit(1, 1)
qc_y_meas.h(0)
qc_y_meas.x(0) # Apply the noisy, twirled gate
qc_y_meas.append(SdgGate(), [0])
qc_y_meas.h(0)
# Measure will be added by sample_circuit

# Circuit 3: Prepare |+>, apply X, measure <Z>
qc_z_meas = QuantumCircuit(1, 1)
qc_z_meas.h(0)
qc_z_meas.x(0) # Apply the noisy, twirled gate
# Measure will be added by sample_circuit

circuits = {'X': qc_x_meas, 'Y': qc_y_meas, 'Z': qc_z_meas}
transpiled_circs = {}

print(f"Testing on {backend.name} ({n_qubits_total} qubits)")
for name, qc in circuits.items():
    # Transpile for the backend. This maps our 1 qubit to one of the 5.
    layout = [0]
    t_qc = transpile(qc, basis_gates=basis_gates, optimization_level=0, initial_layout=layout)
    transpiled_circs[name] = t_qc


# --- 2. Run the UN-TWIRLED Test ---
print("\n--- Running UN-TWIRLED Test (non-Pauli channel) ---")
expvals_un_twirled = {}
for name, t_qc in transpiled_circs.items():
    # Get the 2^5 distribution
    dist = sample_circuit(t_qc, backend, shots=8192)
    print(dist)
    # Get the <Z> of the *measurement basis* (which corresponds to <X>, <Y>, or <Z>)
    expval = get_marginal_expval_z(dist, n_qubits_total, 0)
    expvals_un_twirled[name] = expval
    print(f"  < {name} >_out = {expval:.6f}")


# --- 3. Run the TWIRLED Test ---
print("\n--- Running TWIRLED Test (Pauli-fied channel) ---")
expvals_twirled = {}
for name, qc in circuits.items(): # circuits contains the LOGICAL circuits
    
    # Call the new "manager" function
    dist = get_full_twirled_distribution(
        qc,  # Pass the 1-qubit logical circuit
        backend,
        basis_gates,
        shots=1024, # A decent number of total shots
        num_gate_randomizations=20, # 20 gate twirls
        num_meas_randomizations=20  # 20 meas twirls 
    )
    
    # (The rest of your test is unchanged)
    expval = get_marginal_expval_z(dist, n_qubits_total, 0)
    expvals_twirled[name] = expval
    print(f"  < {name} >_out = {expval:.6f}")

Testing on fake_athens (5 qubits)

--- Running UN-TWIRLED Test (non-Pauli channel) ---
[0.99743652 0.00256348]
  < X >_out = 0.994873
[0.51147461 0.48852539]
  < Y >_out = 0.022949
[0.49926758 0.50073242]
  < Z >_out = -0.001465

--- Running TWIRLED Test (Pauli-fied channel) ---
Full twirling complete.
  < X >_out = 0.977843
Full twirling complete.
  < Y >_out = -0.007157
Full twirling complete.
  < Z >_out = -0.006471
